In [18]:
import os, sys
sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django import forms
from edc_constants.constants import YES, POS
from flourish_caregiver.helper_classes import MaternalStatusHelper, EnrollmentHelper

In [19]:
def enrollment_hiv_status(subject_identifier, child_subject_identifier=None):
    try:
        previous_enrollment = CaregiverPreviouslyEnrolled.objects.get(
            subject_identifier=subject_identifier)
    except CaregiverPreviouslyEnrolled.DoesNotExist:
        query_options = {'subject_identifier': subject_identifier,}
        if child_subject_identifier:
            query_options.append(child_subject_identifier=child_subject_identifier)
        try:
            antenatal_enrollment = AntenatalEnrollment.objects.filter(
                **query_options).latest('report_datetime')
        except AntenatalEnrollment.DoesNotExist:
            # To refactor to include new enrollees
            pass
        else:
            enrollment_helper = EnrollmentHelper(
                instance_antenatal=antenatal_enrollment,
                exception_cls=forms.ValidationError)
            try:
                return enrollment_helper.enrollment_hiv_status
            except ValidationError:
                return UNK
    else:
        if previous_enrollment.current_hiv_status is not None:
            return previous_enrollment.current_hiv_status

    maternal_dataset_objs = None
    locator_obj = None

    if subject_identifier:
        locator_obj = get_locator_model_obj(subject_identifier)

    study_maternal_identifier = getattr(
        locator_obj, 'study_maternal_identifier', None) or study_maternal_identifier

    if study_maternal_identifier:
        maternal_dataset_objs = MaternalDataset.objects.filter(
            study_maternal_identifier=study_maternal_identifier)

    # for maternal_dataset_obj in maternal_dataset_objs:
    if maternal_dataset_objs:
        mom_hiv_statuses = maternal_dataset_objs.values_list(
            'mom_hivstatus', flat=True)

        if 'HIV-infected' in mom_hiv_statuses:
            return POS
        else:
            return NEG

    return UNK

In [20]:
# Adults, who:
#     1. Are on study
#     2. Who indicated on the most recent consent form they are willing to be contacted for future studies
#     3. Both biological mothers and caregivers
#     4. If biological mother, they must have been living with HIV during the birth of their child on FLOURISH
#     5. Who have completed the HIV Disclosure CRF
#     ; by “Disclosed to their Child” “Have Not Disclosed to their Child but Intends to”,
#     and “Have Not Disclosed to their Child and does not intend to
#         * This can be pulled by using Q1 and Q2 on the HIV Disclosure CRF

In [21]:
offstudy_idx = CaregiverOffStudy.objects.values_list('subject_identifier', flat=True) # (1)

consents = SubjectConsent.objects.exclude(
    subject_identifier__in=offstudy_idx).values_list('subject_identifier', flat=True)

consents = set(consents)
caregiver_idxs = []

for sidx in consents:
    latest_consent = SubjectConsent.objects.filter(subject_identifier=sidx).latest('consent_datetime')
    if latest_consent.future_contact == YES: # (2)
        is_prev_study = latest_consent.get_maternal_dataset()
        is_biological = latest_consent.biological_caregiver
        if not is_prev_study and is_biological: # (4)
            enrolment_hiv_status = enrollment_hiv_status(subject_identifier=sidx)
            if enrolment_hiv_status != POS:
                enrol_tests = HIVRapidTestCounseling.objects.filter(
                    maternal_visit__subject_identifier=sidx,
                    maternal_visit__visit_code='1000M',
                    rapid_test_done=YES)
                if not enrol_tests.exists():
                    continue
                results = []
                for enrol_test in enrol_tests:
                    results.append(enrol_test.result != POS)
                if all(results):
                    continue

            disclosure_crfs = [HIVDisclosureStatusA, HIVDisclosureStatusB, HIVDisclosureStatusC]
            disclosure_exists = False
            for model_cls in disclosure_crfs:
                _disclosure_exists = model_cls.objects.filter(
                    maternal_visit__subject_identifier=sidx, disclosed_status__isnull=False).exists()
                if _disclosure_exists: # (5)
                    disclosure_exists = True
                    break
            if not disclosure_exists:
                continue
        caregiver_idxs.append({'subject_identifier': sidx})

In [22]:
import csv
keys = caregiver_idxs[0].keys()
with open('disc_caregiver_pids.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(caregiver_idxs)

In [23]:
# Adolescents who:
#     1. Are on study
#     2. Caregiver indicated on the most recent consent form they are willing to be contacted
#        for future studies
#     3. Mother or Caregiver, indicated they were disclosed to (using the HIV Disclosure CRF)
#     4. Their bio mother must have been living with HIV when they were in-utero
#     5. Child’s age

In [24]:
child_offstudy_idx = ChildOffStudy.objects.values_list('subject_identifier', flat=True) # (1)

child_consents = CaregiverChildConsent.objects.exclude(
    subject_identifier__in=child_offstudy_idx).values_list('subject_identifier', flat=True)

child_consents = set(child_consents)
child_idxs = []

for sidx in child_consents:
    latest_consent = CaregiverChildConsent.objects.filter(subject_identifier=sidx).latest('consent_datetime')
    caregiver_consent = latest_consent.subject_consent
    if caregiver_consent.future_contact == YES: # (2)
        disclosure_crfs = [HIVDisclosureStatusA, HIVDisclosureStatusB, HIVDisclosureStatusC]
        disclosed = False
        for model_cls in disclosure_crfs:
            disclosure_exists = model_cls.objects.filter(
                maternal_visit__subject_identifier=caregiver_consent.subject_identifier,
                associated_child_identifier=sidx,
                disclosed_status=YES).exists()
            if disclosure_exists: # (3)
                disclosed = True
        if not disclosed:
            continue
        enrolment_hiv_status = MaternalStatusHelper(
            subject_identifier=caregiver_consent.subject_identifier).enrollment_hiv_status
        if enrolment_hiv_status != POS: # (4)
            if not bool(latest_consent.study_child_identifier):
                try:
                    enrol_test = HIVRapidTestCounseling.objects.get(
                        maternal_visit__subject_identifier=caregiver_consent.subject_identifier,
                        maternal_visit__visit_code='1000M',
                        rapid_test_done=YES)
                except HIVRapidTestCounseling.DoesNotExist:
                    continue
                else:
                    if enrol_test.result != POS:
                        continue
            else:
                continue
        child_idxs.append({'subject_identifier': sidx})

In [25]:
import csv
keys = child_idxs[0].keys()
with open('disc_child_pids.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(child_idxs)